# Colombian Constitution EDA.

Brief resume about the methodology and the results that we got.

## About the Document

Little research about the context of the document.

## DS Pipeline



## Document Data Model

PowerPoint graph, meanwhile we can use the hierarchy of the articles dictionary

```
hierarchy = {
    'TITULO' : 'h1',
    'DISPOSICIONES' : 'h1',
    'CAPITULO' : 'h2',
    'ARTÍCULO' : 'p'

}
```

## EDA

### 1) Labels and quantity


In [43]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math

# Importing dataset

In [44]:
#import Dataset
df = pd.read_json('../constitucion_col_dict.json')

In [45]:
print("Shape of data=>",df.shape)
df.head(5)

Shape of data=> (439, 3)


,h1,h2,article
0,"{'head': 'TITULO I ', 'name': 'DE LOS PRINCIPI...","{'head': None, 'name': None}","{'name': 'Artículo 1', 'content': ['Artículo 1..."
1,"{'head': 'TITULO I ', 'name': 'DE LOS PRINCIPI...","{'head': None, 'name': None}","{'name': 'Artículo 2', 'content': ['Artículo 2..."
2,"{'head': 'TITULO I ', 'name': 'DE LOS PRINCIPI...","{'head': None, 'name': None}","{'name': 'Artículo 3', 'content': ['Artículo 3..."
3,"{'head': 'TITULO I ', 'name': 'DE LOS PRINCIPI...","{'head': None, 'name': None}","{'name': 'Artículo 4', 'content': ['Artículo 4..."
4,"{'head': 'TITULO I ', 'name': 'DE LOS PRINCIPI...","{'head': None, 'name': None}","{'name': 'Artículo 5', 'content': ['Artículo 5..."


In [46]:
df.isnull().sum()

h1         0
h2         0
article    0
dtype: int64

In [67]:
h1=df['h1'].apply(pd.Series)
h1.rename(columns={"head": "Titulos","name":"NombreTitulo"},inplace=True)
h2=df['h2'].apply(pd.Series)
h2.rename(columns={"head": "Capitulos","name":"NombreCapitulo"},inplace=True)
article=df['article'].apply(pd.Series)
article.rename(columns={"name": "Articulos","content":"Contenido"},inplace=True)

In [73]:
print("Shape of data=>",h1.shape)
print("Shape of data=>",h2.shape)
print("Shape of data=>",article.shape)

Shape of data=> (439, 2)
Shape of data=> (439, 2)
Shape of data=> (439, 2)


In [93]:
frames = [h1, h2, article]
df=pd.concat(frames, axis=1)
df.replace(r'\n',' ', regex=True,inplace=True) 
df['Contenido'] = df['Contenido'].str.join(',')

In [94]:
df.head(15)

,Titulos,NombreTitulo,Capitulos,NombreCapitulo,Articulos,Contenido
0,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 1,Artículo 1. Colombia es un Estado social de de...
1,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 2,Artículo 2. Son fines esenciales del Estado: s...
2,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 3,Artículo 3. La soberanía reside exclusivamente...
3,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 4,Artículo 4. La Constitución es norma de normas...
4,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 5,"Artículo 5. El Estado reconoce, sin discrimina..."
5,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 6,Artículo 6. Los particulares sólo son responsa...
6,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 7,Artículo 7. El Estado reconoce y protege la di...
7,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 8,Artículo 8. Es obligación del Estado y de las ...
8,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 9,Artículo 9. Las relaciones exteriores del Esta...
9,TITULO I,DE LOS PRINCIPIOS FUNDAMENTALES,None,None,Artículo 10,Artículo 10. El castellano es el idioma oficia...


In [95]:
print("Shape of data=>",df.shape)

Shape of data=> (439, 6)


In [105]:
df.set_index(['Titulos','Capitulos','Articulos'],inplace=True)
df.sort_index(inplace=True)

In [113]:
df.head(20)

NombreTitulo  \
Titulos                     Capitulos   Articulos                              
DISPOSICIONES TRANSITORIAS  CAPITULO 1  Artículo transitorio 1          None   
                                        Artículo transitorio 10         None   
                                        Artículo transitorio 11         None   
                                        Artículo transitorio 12         None   
                                        Artículo transitorio 13         None   
                                        Artículo transitorio 14         None   
                                        Artículo transitorio 15         None   
                                        Artículo transitorio 16         None   
                                        Artículo transitorio 17         None   
                                        Artículo transitorio 18         None   
                                        Artículo transitorio 2          None   
                                        Artículo transitorio 3          None   
                                        Artículo transitorio 4          None   
                                        Artículo transitorio 5          None   
                                        Artículo transitorio 6          None   
                                        Artículo transitorio 7          None   
                                        Artículo transitorio 8          None   
                                        Artículo transitorio 9          None   
                            CAPITULO 2  Artículo transitorio 20         None   
                                        Artículo transitorio 21         None   

                                                                NombreCapitulo  \
Titulos                     Capitulos   Articulos                                
DISPOSICIONES TRANSITORIAS  CAPITULO 1  Artículo transitorio 1            None   
                                        Artículo transitorio 10           None   
                                        Artículo transitorio 11           None   
                                        Artículo transitorio 12           None   
                                        Artículo transitorio 13           None   
                                        Artículo transitorio 14           None   
                                        Artículo transitorio 15           None   
                                        Artículo transitorio 16           None   
                                        Artículo transitorio 17           None   
                                        Artículo transitorio 18           None   
                                        Artículo transitorio 2            None   
                                        Artículo transitorio 3            None   
                                        Artículo transitorio 4            None   
                                        Artículo transitorio 5            None   
                                        Artículo transitorio 6            None   
                                        Artículo transitorio 7            None   
                                        Artículo transitorio 8            None   
                                        Artículo transitorio 9            None   
                            CAPITULO 2  Artículo transitorio 20           None   
                                        Artículo transitorio 21           None   

                                                                                                         Contenido  
Titulos                     Capitulos   Articulos                                                                   
DISPOSICIONES TRANSITORIAS  CAPITULO 1  Artículo transitorio 1   Artículo transitorio 1. Convócase a elecciones...  
                                        Artículo transitorio 10  Artículo transitorio 10. Los decretos que expi...  
                              

In [112]:
df.tail(20)

NombreTitulo  \
Titulos      Capitulos   Articulos                                                       
TITULO XII   CAPITULO 4  Artículo 361  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 362  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 363  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 364  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
             CAPITULO 5  Artículo 365  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 366  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 367  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 368  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 369  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 370  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
             CAPITULO 6  Artículo 371  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 372  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
                         Artículo 373  DEL REGIMEN ECONOMICO Y DE LA HACIENDA PUBLICA    
TITULO XIII  NaN         Artículo 374                DE LA REFORMA DE LA CONSTITUCION    
                         Artículo 375                DE LA REFORMA DE LA CONSTITUCION    
                         Artículo 376                DE LA REFORMA DE LA CONSTITUCION    
                         Artículo 377                DE LA REFORMA DE LA CONSTITUCION    
                         Artículo 378                DE LA REFORMA DE LA CONSTITUCION    
                         Artículo 379                DE LA REFORMA DE LA CONSTITUCION    
                         Artículo 380                DE LA REFORMA DE LA CONSTITUCION    

                                                                          NombreCapitulo  \
Titulos      Capitulos   Articulos                                                         
TITULO XII   CAPITULO 4  Artículo 361  DE LA DISTRIBUCION DE RECURSOS Y DE LAS COMPET...   
                         Artículo 362  DE LA DISTRIBUCION DE RECURSOS Y DE LAS COMPET...   
                         Artículo 363  DE LA DISTRIBUCION DE RECURSOS Y DE LAS COMPET...   
                         Artículo 364  DE LA DISTRIBUCION DE RECURSOS Y DE LAS COMPET...   
             CAPITULO 5  Artículo 365  DE LA FINALIDAD SOCIAL DEL ESTADO Y DE LOS SER...   
                         Artículo 366  DE LA FINALIDAD SOCIAL DEL ESTADO Y DE LOS SER...   
                         Artículo 367  DE LA FINALIDAD SOCIAL DEL ESTADO Y DE LOS SER...   
                         Artículo 368  DE LA FINALIDAD SOCIAL DEL ESTADO Y DE LOS SER...   
                         Artículo 369  DE LA FINALIDAD SOCIAL DEL ESTADO Y DE LOS SER...   
                         Artículo 370  DE LA FINALIDAD SOCIAL DEL ESTADO Y DE LOS SER...   
             CAPITULO 6  Artículo 371                               DE LA BANCA CENTRAL    
                         Artículo 372                               DE LA BANCA CENTRAL    
                         Artículo 373                               DE LA BANCA CENTRAL    
TITULO XIII  NaN         Artículo 374                                               None   
                         Artículo 375                                               None   
                         Artículo 376                                               None   
                         Artículo 377                                               None   
                         Artículo 378                                               None   
                         Artículo 379                                               None   
                         Artículo 380                                               None   

                                                                    

# Funcion que cuente h1, h2 y articulos

### 2) Number of paragraphs

### 3) Number of words

### 4) Lematizacion y que pasa despues

### 5) Most frequent relevant words

## TFIDF / NMF / LDA